In [1]:
import numpy as np
import xarray as xr

# Load environmental dust cube
cube = np.load("../outputs/environmental_dust_cube.npy")

# Create xarray dataset
dust_ds = xr.Dataset(
    {
        "NDVI": (("time", "y", "x"), cube[..., 0]),
        "Wind Speed": (("time", "y", "x"), cube[..., 1]),
        "Wind Direction": (("time", "y", "x"), cube[..., 2]),
        "Dust Activity Index": (("time", "y", "x"), cube[..., 3]),
    },
    coords={
        "time": np.arange(cube.shape[0]),
        "y": np.arange(cube.shape[1]),
        "x": np.arange(cube.shape[2]),
    }
)

print("dust_ds loaded successfully")
print(dust_ds)


dust_ds loaded successfully
<xarray.Dataset> Size: 233MB
Dimensions:              (time: 14, y: 721, x: 1440)
Coordinates:
  * time                 (time) int32 56B 0 1 2 3 4 5 6 7 8 9 10 11 12 13
  * y                    (y) int32 3kB 0 1 2 3 4 5 6 ... 715 716 717 718 719 720
  * x                    (x) int32 6kB 0 1 2 3 4 5 ... 1435 1436 1437 1438 1439
Data variables:
    NDVI                 (time, y, x) float32 58MB 0.5 0.5 0.5 ... 0.5 0.5 0.5
    Wind Speed           (time, y, x) float32 58MB 6.318 6.318 ... 4.691 4.691
    Wind Direction       (time, y, x) float32 58MB 1.222 1.222 ... -0.3949
    Dust Activity Index  (time, y, x) float32 58MB 3.159 3.159 ... 2.345 2.345


In [15]:
pip install dash-bootstrap-components

  Using cached dash_bootstrap_components-2.0.4-py3-none-any.whl.metadata (18 kB)
Using cached dash_bootstrap_components-2.0.4-py3-none-any.whl (204 kB)
Note: you may need to restart the kernel to use updated packages.


In [21]:
# =========================
# LOAD MODEL PREDICTIONS
# =========================
import numpy as np
import xarray as xr

# Load ConvLSTM predictions
pred_array = np.load("../outputs/convlstm_predictions.npy")

predicted_ds = xr.Dataset(
    {
        var: (("time", "y", "x"), pred_array)
        for var in dust_ds.data_vars
    },
    coords=dust_ds.coords
)

print("ConvLSTM predictions loaded")


FileNotFoundError: [Errno 2] No such file or directory: '../outputs/convlstm_predictions.npy'

In [ ]:
from dash import Dash, dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import webbrowser
import threading
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# =========================
# PREPARE DATA
# =========================
predicted_ds = dust_ds.shift(time=-1)

# =========================
# ANALYTICAL FUNCTIONS
# =========================
def classify_risk(data):
    mean_val = np.nanmean(data)
    if mean_val < 0.3: return "LOW", "#00FFC8", "🟢"
    elif mean_val < 0.6: return "MODERATE", "#FFB300", "🟡"
    else: return "HIGH", "#FF4B4B", "🔴"

def compute_uncertainty(series, window=5):
    return np.array([np.std(series[max(0, i-window):i+1]) for i in range(len(series))])

def detect_hotspots(data, threshold=0.7):
    return (data > threshold).sum() / data.size * 100

def health_impact(risk):
    if risk == "LOW": return "Minimal respiratory impact expected."
    elif risk == "MODERATE": return "Sensitive groups may experience discomfort."
    else: return "High risk for respiratory issues. Precaution advised."

def compute_metrics(obs, pred):
    mae = mean_absolute_error(obs.flatten(), pred.flatten())
    rmse = np.sqrt(mean_squared_error(obs.flatten(), pred.flatten()))
    return mae, rmse

# =========================
# CREATE DASH APP
# =========================
app = Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])

app.index_string = '''
<!DOCTYPE html>
<html>
<head>
    {%metas%}<title>GEO-DUST ANALYTICS</title>{%css%}
    <style>
        body { background-color: #080a0e; overflow-x: hidden; }
        .panel { background: rgba(25,28,36,0.85); border-radius: 15px; padding: 30px; border: 1px solid rgba(255,255,255,0.1); }
        .kpi-card { background: #11141d; border-radius: 12px; padding: 20px; border: 1px solid #2d3039; text-align: center; }
        .kpi-val { font-size: 22px; font-weight: bold; margin-bottom: 0; }
        .kpi-label { font-size: 11px; color: #888; text-transform: uppercase; letter-spacing: 1px; }
        /* Professional Dropdown Styling */
        .dash-dropdown { z-index: 1050 !important; position: relative; }
        .dash-dropdown .Select-control { background-color: white !important; color: black !important; }
        .dash-dropdown .is-focused:not(.is-open) > .Select-control { background-color: white !important; }
        .section-spacer { margin-bottom: 35px; }
    </style>
</head>
<body>{%app_entry%}{%config%}{%scripts%}{%renderer%}</body>
</html>
'''

# =========================
# DASHBOARD LAYOUT
# =========================
app.layout = dbc.Container(fluid=True, style={"padding": "40px"}, children=[

    # --- HEADER ---
    dbc.Row(className="section-spacer", children=[
        dbc.Col(width=12, children=[
            html.Div(className="text-center", children=[
                html.H1("GEO-DUST ANALYTICS", 
                        style={"fontWeight": "900", "letterSpacing": "5px", "color": "white", "marginBottom": "5px"}),
                html.P("Predictive Analytics & Spatio-Temporal Risk Engine", 
                        style={"fontSize": "14px", "color": "#58a6ff", "textTransform": "uppercase", "letterSpacing": "2px", "fontWeight": "bold"})
            ])
        ])
    ]),

    # --- TOP KPI ROW ---
    dbc.Row(className="section-spacer", children=[
        dbc.Col(dbc.Card(className="kpi-card", children=[
            html.P("Current Risk", className="kpi-label"),
            html.Div(id="kpi-risk", className="kpi-val")
        ])),
        dbc.Col(dbc.Card(className="kpi-card", children=[
            html.P("Hotspot Area", className="kpi-label"),
            html.Div(id="kpi-hotspot", className="kpi-val", style={"color": "#00FFC8"})
        ])),
        dbc.Col(dbc.Card(className="kpi-card", children=[
            html.P("Mean Intensity", className="kpi-label"),
            html.Div(id="kpi-mean", className="kpi-val", style={"color": "white"})
        ])),
        dbc.Col(dbc.Card(className="kpi-card", children=[
            html.P("Model Performance", className="kpi-label"),
            html.Div(id="kpi-mae", className="kpi-val", style={"color": "#58a6ff"})
        ])),
    ]),

    dbc.Row([
        # --- LEFT SIDEBAR ---
        dbc.Col(width=3, children=[
            html.Div(className="panel", children=[
                html.Label("Variable Select", className="text-info font-weight-bold"),
                dcc.Dropdown(
                    id="variable-dropdown",
                    options=[{"label": v, "value": v} for v in dust_ds.data_vars],
                    value=list(dust_ds.data_vars)[0],
                    className="dash-dropdown mb-4",
                    clearable=False
                ),

                html.Label("Analysis Mode", className="text-info font-weight-bold"),
                dbc.RadioItems(
                    id="data-mode",
                    options=[
                        {"label": "Observed Data", "value": "obs"},
                        {"label": "Model Prediction", "value": "pred"},
                        {"label": "Variance Analysis", "value": "diff"},
                    ],
                    value="obs",
                    className="mb-4",
                    style={"color": "#CCCCCC"}
                ),

                html.Label("Forecast Horizon", className="text-info font-weight-bold"),
                dcc.Dropdown(
                    id="forecast-horizon",
                    options=[{"label": f"+{h} Steps", "value": h} for h in [1, 3, 6]],
                    value=1,
                    className="dash-dropdown mb-4",
                    clearable=False
                ),

                html.Label("Timeline Control", className="text-info font-weight-bold"),
                dbc.Button("▶ Play / ⏸ Pause", id="play-button", color="outline-info", size="sm", className="mb-3 w-100"),
                dcc.Slider(
                    id="time-slider",
                    min=0, max=len(dust_ds.time) - 1, step=1, value=0,
                    tooltip={"always_visible": True},
                ),
                dcc.Interval(id="auto-stepper", interval=1200, disabled=True),

                html.Hr(style={"borderColor": "#444"}),
                html.Div(id="health-indicator", style={"fontSize": "13px", "color": "#CCCCCC", "textAlign": "center"}),
                
                dbc.Button("📥 Export Insights", id="export-btn", color="secondary", size="sm", className="mt-4 w-100"),
            ])
        ]),

        # --- RIGHT MAIN PANEL ---
        dbc.Col(width=9, children=[
            html.Div(style={"background": "#11141d", "padding": "15px", "borderRadius": "15px", "border": "1px solid #2d3039", "marginBottom": "25px"}, children=[
                dcc.Graph(id="spatial-map", style={"height": "55vh"}, config={"displayModeBar": False}),
            ]),
            
            dbc.Row(children=[
                dbc.Col(width=6, children=[
                    html.Div(style={"background": "#11141d", "padding": "15px", "borderRadius": "15px", "border": "1px solid #2d3039"}, children=[
                        dcc.Graph(id="time-series", style={"height": "25vh"}, config={"displayModeBar": False})
                    ])
                ]),
                dbc.Col(width=6, children=[
                    html.Div(style={"background": "#11141d", "padding": "15px", "borderRadius": "15px", "border": "1px solid #2d3039"}, children=[
                        dcc.Graph(id="cross-section", style={"height": "25vh"}, config={"displayModeBar": False})
                    ])
                ])
            ])
        ])
    ])
])

# =========================
# CALLBACKS
# =========================

@app.callback(
    Output("auto-stepper", "disabled"),
    Output("play-button", "children"),
    Input("play-button", "n_clicks"),
    State("auto-stepper", "disabled")
)
def toggle_play(n, disabled):
    if n is None: return True, "▶ Play / ⏸ Pause"
    return not disabled, "⏸ Pause" if disabled else "▶ Play"

@app.callback(
    Output("time-slider", "value"),
    Input("auto-stepper", "n_intervals"),
    State("time-slider", "value"),
    State("time-slider", "max")
)
def auto_step(n, current, max_val):
    return 0 if current >= max_val else current + 1

@app.callback(
    [Output("spatial-map", "figure"),
     Output("kpi-risk", "children"),
     Output("kpi-risk", "style"),
     Output("kpi-hotspot", "children"),
     Output("kpi-mean", "children"),
     Output("kpi-mae", "children"),
     Output("health-indicator", "children")],
    [Input("variable-dropdown", "value"),
     Input("time-slider", "value"),
     Input("data-mode", "value"),
     Input("forecast-horizon", "value")]
)
def update_spatial_view(variable, t, mode, horizon):
    future_t = min(t + horizon, len(dust_ds.time)-1)
    observed = dust_ds[variable].isel(time=t).values
    predicted = predicted_ds[variable].isel(time=future_t).values

    if mode == "obs": data, title = observed, "Observed"
    elif mode == "pred": data, title = predicted, "Forecasted"
    else: data, title = predicted - observed, "Variance"

    # Color Mapping Logic
    if mode == "diff":
        color_scale = "RdBu_r"
    elif "wind" in variable.lower():
        color_scale = "Cividis" # Professional blue-yellow for wind
    else:
        color_scale = "Inferno" # Intensity for dust

    risk, color, icon = classify_risk(data)
    hotspot_pct = detect_hotspots(data)
    mae, rmse = compute_metrics(observed, predicted)
    
    fig = px.imshow(data, origin="upper", color_continuous_scale=color_scale)
    fig.update_layout(
        title=dict(text=f"<b>{title} {variable} Analysis</b> | Step: {t}", x=0.05),
        template="plotly_dark", 
        margin=dict(l=0, r=0, t=40, b=0),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        coloraxis_colorbar=dict(thickness=15, title="")
    )
    fig.update_xaxes(visible=False).update_yaxes(visible=False)

    return fig, f"{icon} {risk}", {"color": color}, f"{hotspot_pct:.1f}%", f"{np.nanmean(data):.3f}", f"MAE: {mae:.3f}", health_impact(risk)

@app.callback(
    [Output("time-series", "figure"),
     Output("cross-section", "figure")],
    [Input("spatial-map", "clickData"),
     Input("variable-dropdown", "value"),
     Input("time-slider", "value")]
)
def update_secondary_graphs(clickData, variable, t):
    x, y = (dust_ds.dims["x"]//2, dust_ds.dims["y"]//2)
    if clickData:
        x, y = int(clickData["points"][0]["x"]), int(clickData["points"][0]["y"])

    # Time Series (Original Uncertainty bands preserved)
    hist = dust_ds[variable][:, y, x].values
    pred = predicted_ds[variable][:, y, x].values
    unc = compute_uncertainty(pred)

    ts_fig = go.Figure()
    ts_fig.add_trace(go.Scatter(y=hist, name="Observed", line=dict(color="#00FFC8", width=2)))
    ts_fig.add_trace(go.Scatter(y=pred, name="Forecast", line=dict(color="#FF4B4B", dash="dot")))
    ts_fig.add_trace(go.Scatter(y=pred+unc, showlegend=False, line=dict(width=0)))
    ts_fig.add_trace(go.Scatter(y=pred-unc, fill="tonexty", fillcolor="rgba(255,75,75,0.2)", line=dict(width=0), name="Uncertainty"))
    ts_fig.update_layout(title=f"Temporal Trend at Grid ({x}, {y})", template="plotly_dark", margin=dict(l=40, r=10, t=30, b=30), paper_bgcolor='rgba(0,0,0,0)')

    # Cross Section
    y_slice = dust_ds[variable].isel(time=t, y=y).values
    cs_fig = go.Figure(go.Scatter(y=y_slice, fill="tozeroy", line=dict(color="#FFB300"), name="Intensity"))
    cs_fig.update_layout(title=f"Latitudinal Slice (Index Y={y})", template="plotly_dark", margin=dict(l=40, r=10, t=30, b=30), paper_bgcolor='rgba(0,0,0,0)')

    return ts_fig, cs_fig

# =========================
# AUTO OPEN BROWSER & RUN
# =========================
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8051/")

if __name__ == "__main__":
    threading.Timer(1, open_browser).start()
    app.run(debug=False, port=8051)

C:\Users\Haneesh.B\AppData\Local\Temp\ipykernel_31112\1895202478.py:255: FutureWarning:

The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.

C:\Users\Haneesh.B\AppData\Local\Temp\ipykernel_31112\1895202478.py:255: FutureWarning:

The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.

C:\Users\Haneesh.B\AppData\Local\Temp\ipykernel_31112\1895202478.py:255: FutureWarning:

The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.

C:\Users\Haneesh.B\AppData\Local\Temp

In [ ]:
import webbrowser
import threading

def open_browser():
    webbrowser.open_new("http://127.0.0.1:8051/")

# Open browser 1 second after server starts
threading.Timer(1, open_browser).start()

app.run(debug=False, port=8051)


C:\Users\Haneesh.B\AppData\Local\Temp\ipykernel_31112\1737357104.py:222: FutureWarning:

The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.

C:\Users\Haneesh.B\AppData\Local\Temp\ipykernel_31112\1737357104.py:222: FutureWarning:

The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.

C:\Users\Haneesh.B\AppData\Local\Temp\ipykernel_31112\1737357104.py:222: FutureWarning:

The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.

C:\Users\Haneesh.B\AppData\Local\Temp